In [1]:
from robot_hand import Robot

In [2]:
robot = Robot()
robot.display(robot.q0)

The previously created objects will not be destroyed and do not have to be created again.


/home/cryscan/IdeaProjects/to-ihc/display.py:53: DeprecatedWarning: Call to deprecated function se3ToXYZQUATtuple. This function is now called SE3ToXYZQUATtuple. Please change for this new signature to delete this warning.
  pin.se3ToXYZQUATtuple(M))


In [3]:
from quadprog import solve_qp

import numpy as np
from numpy.random import rand

In [4]:
import pinocchio as se3
q = robot.q0
q_des = rand(robot.model.nq)

vq = rand(robot.model.nv)
aq0 = np.zeros(robot.model.nv)

In [5]:
# Create 10 witness points in the rendering window
for i in range(10):
    robot.viewer.viewer.gui.addCylinder('world/wa%i' % i, .01, .003, [1, 0, 0, 1])
    robot.viewer.viewer.gui.addCylinder('world/wb%i' % i, .01, .003, [1, 0, 0, 1])
    robot.viewer.viewer.gui.setVisibility('world/wa%i' % i, 'OFF')
    robot.viewer.viewer.gui.setVisibility('world/wb%i' % i, 'OFF')
# Add 4 pairs between finger tips and palm
robot.collisionPairs.append([2, 8])
robot.collisionPairs.append([2, 11])
robot.collisionPairs.append([2, 14])
robot.collisionPairs.append([2, 16])
# Compute distance between object 2 and 8, i.e the first collision pair
idx = 0
dist = robot.checkCollision(idx)
# Display the collision pair by adding two disks at the witness points.
robot.displayCollision(idx, 0)

/home/cryscan/IdeaProjects/to-ihc/robot_hand.py:100: DeprecatedWarning: Call to deprecated function cross. Please use numpy.cross(a, b) or numpy.cross(a, b, axis=0).
  r1 = cross(ab, x)
/home/cryscan/IdeaProjects/to-ihc/robot_hand.py:103: DeprecatedWarning: Call to deprecated function cross. Please use numpy.cross(a, b) or numpy.cross(a, b, axis=0).
  r1 = cross(ab, x)
/home/cryscan/IdeaProjects/to-ihc/robot_hand.py:105: DeprecatedWarning: Call to deprecated function cross. Please use numpy.cross(a, b) or numpy.cross(a, b, axis=0).
  r2 = cross(ab, r1)


In [ ]:
while True:
    # compute dynamic drift -- Coriolis, centrifugal, gravity
    b = se3.rnea(robot.model, robot.data, q, vq, aq0)
    # compute mass matrix M
    Minv = se3.computeMinverse(robot.model, robot.data, q)

    Kv = 1
    Kp = Kv**2 / 4

    tau = Kp * (q_des - q) - Kv * vq
    aq = Minv @ (tau - b)

    dt = 0.01
    vq += aq * dt
    q = se3.integrate(robot.model, q, vq * dt)
    robot.display(q)

In [ ]:
help(se3.computeJointJacobian)